In [7]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc
from wordcloud import WordCloud
import base64
from io import BytesIO
from PIL import Image

# Load data
data = pd.read_csv('C:/Users/cheng/Downloads/dataSample.csv')

# Preprocess data
tone_counts = data['tone'].value_counts().reset_index()
tone_counts.columns = ['tone', 'count']

keyword_columns = ['keyword1', 'keyword2', 'keyword3', 'keyword4', 'keyword5']
keywords = data.melt(value_vars=keyword_columns, var_name='keyword_position', value_name='keyword')
keywords = keywords[keywords['keyword'] != 'None']
keyword_counts = keywords['keyword'].value_counts().reset_index()
keyword_counts.columns = ['keyword', 'count']

# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(zip(keyword_counts['keyword'], keyword_counts['count'])))
wordcloud_image = wordcloud.to_image()

# Save word cloud image to a BytesIO object
img_buffer = BytesIO()
wordcloud_image.save(img_buffer, format='PNG')
img_buffer.seek(0)
img_base64 = base64.b64encode(img_buffer.getvalue()).decode('utf-8')

# Create Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define layout
app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("Community Interaction Dashboard"), className="text-center my-4")),
    
    dbc.Row([
        dbc.Col(dbc.Button('Tone Chart', id='tone-button', n_clicks=0), md=2),
        dbc.Col(dbc.Button('User Type Chart', id='user-type-button', n_clicks=0), md=2),
        dbc.Col(dbc.Button('Gender Chart', id='gender-button', n_clicks=0), md=2),
        dbc.Col(dbc.Button('Income Level Chart', id='income-level-button', n_clicks=0), md=2),
        dbc.Col(dbc.Button('Age Group Chart', id='age-group-button', n_clicks=0), md=2),
        dbc.Col(dbc.Button('Racial Cultural Affiliation Chart', id='racial-cultural-button', n_clicks=0), md=2)
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Graph(id='tone-chart'), md=6),
        dbc.Col(html.Img(id='wordcloud-image', src='data:image/png;base64,{}'.format(img_base64)), md=6)
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Graph(id='user-type-chart'), md=6),
        dbc.Col(dcc.Graph(id='gender-chart'), md=6)
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Graph(id='income-level-chart'), md=6),
        dbc.Col(dcc.Graph(id='age-group-chart'), md=6)
    ]),
    
    dbc.Row(dbc.Col(dcc.Graph(id='racial-cultural-affiliation-chart')))
])

# Define callback to update charts
@app.callback(
    [Output('tone-chart', 'figure'), 
     Output('user-type-chart', 'figure'),
     Output('gender-chart', 'figure'),
     Output('income-level-chart', 'figure'),
     Output('age-group-chart', 'figure'),
     Output('racial-cultural-affiliation-chart', 'figure')],
    [Input('tone-button', 'n_clicks'),
     Input('user-type-button', 'n_clicks'),
     Input('gender-button', 'n_clicks'),
     Input('income-level-button', 'n_clicks'),
     Input('age-group-button', 'n_clicks'),
     Input('racial-cultural-button', 'n_clicks')]
)
def update_charts(tone_clicks, user_type_clicks, gender_clicks, income_level_clicks, age_group_clicks, racial_cultural_clicks):
    
    # Tone Chart
    tone_fig = px.pie(tone_counts, values='count', names='tone', title='Sentiment Analysis')

    # User Type Chart
    user_type_counts = data['User_Type'].value_counts().reset_index()
    user_type_counts.columns = ['User_Type', 'count']
    user_type_fig = px.pie(user_type_counts, values='count', names='User_Type', title='User Type Distribution')

    # Gender Chart
    gender_counts = data['Gender'].value_counts().reset_index()
    gender_counts.columns = ['Gender', 'count']
    gender_fig = px.pie(gender_counts, values='count', names='Gender', title='Gender Distribution')

    # Income Level Chart
    income_level_counts = data['Income_Level'].value_counts().reset_index()
    income_level_counts.columns = ['Income_Level', 'count']
    income_level_fig = px.pie(income_level_counts, values='count', names='Income_Level', title='Income Level Distribution')

    # Age Group Chart
    age_group_counts = data['Age_Group'].value_counts().reset_index()
    age_group_counts.columns = ['Age_Group', 'count']
    age_group_fig = px.pie(age_group_counts, values='count', names='Age_Group', title='Age Group Distribution')

    # Racial Cultural Affiliation Chart
    racial_cultural_counts = data['Racial_Cultural_Affiliation'].value_counts().reset_index()
    racial_cultural_counts.columns = ['Racial_Cultural_Affiliation', 'count']
    racial_cultural_fig = px.pie(racial_cultural_counts, values='count', names='Racial_Cultural_Affiliation', title='Racial/Cultural Affiliation Distribution')

    return tone_fig, user_type_fig, gender_fig, income_level_fig, age_group_fig, racial_cultural_fig


@app.callback(
    [Output('tone-chart', 'style'),
     Output('user-type-chart', 'style'),
     Output('gender-chart', 'style'),
     Output('income-level-chart', 'style'),
     Output('age-group-chart', 'style'),
     Output('racial-cultural-affiliation-chart', 'style')],
    [Input('tone-button', 'n_clicks'),
     Input('user-type-button', 'n_clicks'),
     Input('gender-button', 'n_clicks'),
     Input('income-level-button', 'n_clicks'),
     Input('age-group-button', 'n_clicks'),
     Input('racial-cultural-button', 'n_clicks')]
)
def show_hide_graphs(tone_clicks, user_type_clicks, gender_clicks, income_level_clicks, age_group_clicks, racial_cultural_clicks):
    tone_style = {'display': 'block' if tone_clicks else 'none'}
    user_type_style = {'display': 'block' if user_type_clicks else 'none'}
    gender_style = {'display': 'block' if gender_clicks else 'none'}
    income_level_style = {'display': 'block' if income_level_clicks else 'none'}
    age_group_style = {'display': 'block' if age_group_clicks else 'none'}
    racial_cultural_style = {'display': 'block' if racial_cultural_clicks else 'none'}

    return tone_style, user_type_style, gender_style, income_level_style, age_group_style, racial_cultural_style


if __name__ == "__main__":
    app.run_server(debug=True)